<a href="https://colab.research.google.com/github/karsarobert/DeepLearning2024/blob/main/11/PTE_DL11_text_generation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Szöveggenerálás RNN-nel

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Ez a bemutató azt mutatja be, hogyan lehet szöveget generálni egy karakteralapú RNN segítségével. Rejtő Jenő írásaiból álló adatkészlettel fog dolgozni, amely ötlet Andrej Karpathy The Unreasonable Effectiveness of Recurrent Neural Networks című könyvéből származik. Adott egy karaktersorozat ebből az adatból, képezzünk ki egy modellt a sorozat következő karakterének ("e") előrejelzésére. A modell ismételt meghívásával hosszabb szövegsorozatok generálhatók.

Megjegyzés: Engedélyezze a GPU-gyorsítást a notebook gyorsabb végrehajtásához. A Colab: Futtatási idő > Futtatási idő típusának módosítása > Hardveres gyorsító > GPU.

A következő a mintakimenet, amikor az ebben a bemutatóban szereplő modell 30 epochán keresztül képzett, és a "A" felszólítással indult:

<pre>
An saját magaszására társal lett van a pitánsággal, könnyedéseket.
A herceg szeme le régen is tűt. Mehes. Fülig Jimmy? Mi ajra, ha Hurcunk Fernántesz! És igen ötök boripán az találgassza, Felség, angol hibetkézik az én nevemben, azt hitte, hogy néhány óráj baj vas, hát de nem hites, előfordul egy erőtel sem mozdult. Pedig a hajón vérn A fiú uralkodásai és lenéssel az illetőkeá.
- Hány és GrAndszervez erted?
Így öreg Wilson Hutchins (az amerikai fűtő már csak azért sem vonállattag érezte, amelyen a pincér aztán ki történt. Fel tudum. Most már közzem (Övig nem ívás, hanem Jiment furcsa, akkor ja mokdanás, mert a brót és fél személyesen ismerem.
- Warins a vőlegverék el - mondja a stamát... Az is a pillanatra! - jegyezte meg közölte?
- Ezent lesz a borízsal állt. - Es osztálybal. Öngyen közt kenyeres tudja, ő a próféte megtoválját, így szólt:
- Integessen Felség, hogy egy nap alatt sok mindent tett, embere! Hozzt, hogy nem keresem tovább vezeteti Bannera. Tevissza kelé krabitány az á

</pre>

Bár néhány mondat nyelvtanilag helyes, a legtöbbnek nincs értelme. A modell nem tanulta meg a szavak jelentését, de fontolja meg:

* A modell karakteralapú. Amikor a képzés elkezdődött, a modell nem tudta, hogyan kell egy szót leírni, vagy hogy a szavak egyáltalán a szöveg egységei.

* Amint azt az alábbiakban bemutatjuk, a modell kis szövegrészleteken (egyenként 100 karakter) tanul, és még mindig képes egy hosszabb, összefüggő szerkezetű szövegsorozatot generálni.

## Setup

### TensorFlow és más könyvtárak importálása

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

import os
os.environ['TF_USE_LEGACY_KERAS']='1'

### Az adatkészlet letöltése

Változtassa meg a következő sort, hogy futtassa ezt a kódot a saját data.c fájlján.

In [ ]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = tf.keras.utils.get_file('piszkosfred.txt', 'https://raw.githubusercontent.com/karsarobert/DeepLearning2024/main/piszkosfred.txt')

349517/349517 [==============================] - 0s 0us/step


### Read the data

Először nézd meg a szöveget:

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 321835 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿
Rejtő Jenő

Piszkos Fred, a kapitány

ELSŐ FEJEZET
1
- Uram! A késemért jöttem!
- Hol hagyta?
- Valami matrózban.
- Milyen kés volt?
- Acél. Keskeny penge, kissé hajlott. Nem látta?
- Várjunk... Csak lassan, kérem... Milyen volt a nyele?
- Kagyló.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

98 unique characters


## A szöveg feldolgozása

### A szöveg vektorizálása

A képzés előtt a karakterláncokat numerikus ábrázolásra kell konvertálni.



In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) #szöveget pontosabban a karaktereket számokká konvertáljuk

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '(' :   3,
  ')' :   4,
  '*' :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '0' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ':' :  19,
  ...
}


In [ ]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\ufeff\nRejtő Jenő\n' ---- characters mapped to int ---- > [97  0 38 51 56 66 90  1 31 51 60 90  0]


### A predikciós feladat

Adott egy karakter vagy egy karaktersorozat, mi a legvalószínűbb következő karakter? Ez az a feladat, amire a modellt betanítjuk. A modell bemenete egy karaktersorozat lesz, és a modellt arra képezzük, hogy minden egyes időlépésnél megjósolja a kimenetet - a következő karaktert.

Mivel az RNN-ek fenntartanak egy belső állapotot, amely a korábban látott elemektől függ, az adott pillanatig kiszámított összes karaktert figyelembe véve, mi a következő karakter?


### Képzési példák és célok létrehozása

Ezután ossza a szöveget példasorozatokra. Minden egyes bemeneti szekvencia a szövegből származó `seq_length` karaktereket tartalmazza.

Minden egyes bemeneti szekvenciához a megfelelő célok ugyanolyan hosszúságú szöveget tartalmaznak, kivéve egy karakterrel jobbra eltolva.

Tehát a szöveget `seq_length+1` hosszúságú darabokra bontjuk. Tegyük fel például, hogy a `seq_length` 4, és a szövegünk a "Hello". A bemeneti szekvencia a "Hell", a célszekvencia pedig az "ello" lenne.

Ehhez először használjuk az `tf.data.Dataset.from_tensor_slices` függvényt, hogy a szövegvektort karakterindexek folyamává alakítsuk.

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)


char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

﻿


R
e
j
t
ő
 
J
e


A "batch" módszerrel ezeket az egyedi karaktereket könnyen átalakíthatja a kívánt méretű szekvenciákká.

Könnyebb látni, hogy mit csinál ez, ha a tokeneket visszacsatoljuk karakterláncokká:

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeff\nRejtő Jenő\n\nPiszkos Fred, a kapitány\n\nELSŐ FEJEZET\n1\n- Uram! A késemért jöttem!\n- Hol hagyta?\n- Val'
'ami matrózban.\n- Milyen kés volt?\n- Acél. Keskeny penge, kissé hajlott. Nem látta?\n- Várjunk... Csak '
'lassan, kérem... Milyen volt a nyele?\n- Kagyló.\n- Hány részből?\n- Egy darabból készült.\n- Akkor nincs'
' baj. Megvan a kés!\n- Hol?\n- A hátamban.\n- Köszönöm...\n- Kérem... A csapos mesélte, hogy milyen szép '
'kés van bennem. Egy darab húszcentis kagy\xadló\xadritkaság.\n- Forduljon meg, kérem, hogy kivegyem...\n- Kit'


A képzéshez szükséged lesz egy `(bemenet, címke)` párokból álló adathalmazra. Ahol a `bemenet` és
`label` szekvenciák. Minden egyes időlépésnél a bemenet az aktuális karakter, a címke pedig a következő karakter.

Íme egy függvény, amely bemenetként egy szekvenciát vesz, duplikálja és eltolja azt, hogy minden egyes időlépésnél összehangolja a bemenetet és a címkét:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text



In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff\nRejtő Jenő\n\nPiszkos Fred, a kapitány\n\nELSŐ FEJEZET\n1\n- Uram! A késemért jöttem!\n- Hol hagyta?\n- Va'
Target data: '\nRejtő Jenő\n\nPiszkos Fred, a kapitány\n\nELSŐ FEJEZET\n1\n- Uram! A késemért jöttem!\n- Hol hagyta?\n- Val'


### Képzési tételek létrehozása

Az `tf.data` segítségével a szöveget kezelhető szekvenciákra osztotta. Mielőtt azonban ezeket az adatokat betáplálnád a modellbe, meg kell keverned az adatokat, és kötegekbe kell csomagolnod őket.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 97 ('\ufeff')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 38 ('R')
Step    2
  input: 38 ('R')
  expected output: 51 ('e')
Step    3
  input: 51 ('e')
  expected output: 56 ('j')
Step    4
  input: 56 ('j')
  expected output: 66 ('t')


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000 #A dataset keveréséhez használt puffer mérete. Minél nagyobb a puffer, annál véletlenszerűbben keverhető az adathalmaz. Segít megelőzni, hogy a modell sorrendiséget tanuljon meg.
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## A modell megépítése

Ez a szakasz a modellt a `keras.Model` alosztályként definiálja (A részletekért lásd [Új rétegek és modellek létrehozása alosztályozással](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

Ez a modell három réteggel rendelkezik:

* `tf.keras.layers.Embedding`: A bemeneti réteg. Egy betanítható keresőtábla, amely minden karakterazonosítót egy `embedding_dim` dimenziójú vektorra képez le;
* `tf.keras.layers.GRU`: Egyfajta RNN, amelynek mérete `units=rnn_units` (Itt egy LSTM réteget is használhatsz.)
* `tf.keras.layers.Dense`: A kimeneti réteg, `vocab_size` kimenetekkel. A szókészlet minden egyes karakterére egy logaritást ad ki. Ezek az egyes karakterek log-valószínűségét adják meg a modell szerint.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

#Ez a kaggleről van

In [ ]:
model = build_model(vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           25088     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 98)            100450    
                                                                 
Total params: 4063842 (15.50 MB)
Trainable params: 4063842 (15.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


A modell minden egyes karakterhez megnézi a beágyazást, lefuttatja a GRU-t egy időlépést a beágyazással, és a sűrű réteget alkalmazza a következő karakter logaritmusát előrejelző logaritmusok létrehozására:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Megjegyzés: A képzéshez használhat egy `keras.Sequential` modellt. A későbbi szöveggeneráláshoz az RNN belső állapotát kell majd kezelned. Egyszerűbb előre felvenni az állapot bemeneti és kimeneti beállításait, mint később átrendezni a modell architektúráját. További részletekért lásd a [Keras RNN útmutató](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Próbálja ki a modellt

Most futtassa a modellt, hogy lássa, a várakozásoknak megfelelően viselkedik-e.

Először ellenőrizze a kimenet alakját:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 98) # (batch_size, sequence_length, vocab_size)


A fenti példában a bemenet szekvencia hossza "100", de a modell bármilyen hosszúságú bemenettel futtatható:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           25088     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 98)            100450    
                                                                 
Total params: 4063842 (15.50 MB)
Trainable params: 4063842 (15.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Ahhoz, hogy tényleges előrejelzéseket kapjunk a modellből, mintát kell vennünk a kimeneti eloszlásból, hogy megkapjuk a tényleges karakterindexeket. Ezt az eloszlást a karakterszókincs logaritmusai határozzák meg.

Megjegyzés: Fontos, hogy ebből az eloszlásból _mintavételezzünk_, mivel az eloszlás _argmax_ értékét véve a modell könnyen hurokba kerülhet.

Próbáljuk ki a tétel első példájánál:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #Véletlenszerűen mintát vesz egy kategorikus eloszlásból
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy() #Eltávolítja a felesleges dimenziót

Ez minden egyes időlépésnél megadja a következő karakterindex előrejelzését:

In [ ]:
sampled_indices

array([90, 80, 45,  1, 67, 10, 18, 86, 23, 83, 42, 49, 81, 69, 38, 72,  9,
       83, 50, 77, 57, 86, 72, 59, 25, 83,  4, 47, 96, 45, 76,  4, 59, 68,
       55, 66, 26, 30, 50, 58, 87, 54, 86, 33,  1, 49, 62, 87, 79, 15, 91,
       38, 72, 43, 67,  4, 12, 75, 70, 59, 87, 69, 97, 40, 22, 64, 15, 65,
       45, 24, 54, 13, 61,  4, 31, 74,  6, 70, 75,  0, 94, 33, 94, 85,  2,
       60,  4,  6,  3, 86, 20, 37, 93, 94, 16, 42, 69, 78, 95, 11])

Dekódolja ezeket, hogy lássa a nem képzett modell által megjósolt szöveget:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' Mit gondol?... Lehetséges az, hogy egy megölt ember visszatérjen?!\n- Ha nem én öltem meg, akkor leh'

Next Char Predictions: 
 'őÜY u19öBéVcàwRz0édÓközmDé)a”YÍ)mvitEIdlúhöL cpúÚ6ŰRzWu)3Éxmúw\ufeffTAr6sYCh4o)JÁ,xÉ\n’L’ó!n),(ö;P‑’7VwÖ“2'


## A modell betanítása

Ezen a ponton a probléma egy szokásos osztályozási problémaként kezelhető. Az előző RNN-állapot és a bemeneti adatok ismeretében ebben az időlépésben meg kell jósolni a következő karakter osztályát.

### Csatoljunk egy optimalizálót és egy veszteségfüggvényt.

A szabványos `tf.keras.losses.sparse_categorical_crossentropy` veszteségfüggvény ebben az esetben működik, mivel az előrejelzések utolsó dimenziójára alkalmazzák.

Mivel a modelled logitokat ad vissza, be kell állítanod a `from_logits` flaget.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.586007, shape=(), dtype=float32)


Egy újonnan inicializált modellnek nem szabad túlságosan biztosnak lennie önmagában, a kimeneti logaritmusoknak mind hasonló nagyságúnak kell lenniük. Ennek megerősítésére ellenőrizheti, hogy az átlagos veszteség exponenciálisa megközelítőleg megegyezik-e a szókincs méretével. Egy sokkal nagyobb veszteség azt jelenti, hogy a modell biztos a rossz válaszaiban, és rosszul van inicializálva:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

98.10194

Konfigurálja a képzési eljárást az `tf.keras.Model.compile` módszerrel. Használja az `tf.keras.optimizers.Adam` modellt alapértelmezett argumentumokkal és a veszteségfüggvényt.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Ellenőrző pontok konfigurálása

A`tf.keras.callbacks.ModelCheckpoint` használatával biztosíthatja, hogy az ellenőrzési pontok a képzés során mentésre kerüljenek:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### A képzés elvégzése

A képzési idő ésszerűségének megőrzése érdekében használjon 10 epochát a modell képzéséhez. A Colabban a gyorsabb képzés érdekében állítsa a futási időt GPU-ra.

In [ ]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
49/49 [==============================] - 8s 67ms/step - loss: 3.5581
Epoch 2/100
49/49 [==============================] - 4s 62ms/step - loss: 2.6558
Epoch 3/100
49/49 [==============================] - 4s 57ms/step - loss: 2.4400
Epoch 4/100
49/49 [==============================] - 3s 58ms/step - loss: 2.3210
Epoch 5/100
49/49 [==============================] - 4s 58ms/step - loss: 2.2171
Epoch 6/100
49/49 [==============================] - 3s 57ms/step - loss: 2.1154
Epoch 7/100
49/49 [==============================] - 3s 58ms/step - loss: 2.0149
Epoch 8/100
49/49 [==============================] - 3s 59ms/step - loss: 1.9200
Epoch 9/100
49/49 [==============================] - 4s 59ms/step - loss: 1.8315
Epoch 10/100
49/49 [==============================] - 3s 58ms/step - loss: 1.7492
Epoch 11/100
49/49 [==============================] - 3s 59ms/step - loss: 1.6777
Epoch 12/100
49/49 [==============================] - 3s 60ms/step - loss: 1.6069
Epoch 13/100
49/49 [=====

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

Hogy ez az előrejelzési lépés egyszerű legyen, használjon 1 tételméretet.

Mivel az RNN állapota időlépésről időlépésre kerül átadásra, a modell csak egy fix kötegméretet fogad el

Ha a modellt más batch_mérettel szeretnénk futtatni, újra kell építenünk a modellt, és vissza kell állítanunk a súlyokat az ellenőrzőpontból.

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

## Szöveg generálása

A legegyszerűbben úgy generálhatsz szöveget ezzel a modellel, ha egy ciklusban futtatod, és a végrehajtás során nyomon követed a modell belső állapotát.

![A szöveg generálásához a modell kimenete visszakerül a bemenetre](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

A modell minden egyes meghívásakor átadunk egy szöveget és egy belső állapotot. A modell a következő karakterre vonatkozó előrejelzést és annak új állapotát adja vissza. A szöveggenerálás folytatásához adja vissza a predikciót és az állapotot.


Az alábbiakban egylépéses előrejelzést teszünk:

Futtassa le egy ciklusban, hogy létrehozzon egy szöveget. Ha megnézzük a generált szöveget, láthatjuk, hogy a modell tudja, mikor kell nagybetűvel írni, bekezdéseket alkotni, és Rejtő-szerű írásszókincset utánoz. A kevés gyakorló epocha miatt még nem tanulta meg, hogy összefüggő mondatokat alkosson.

A kezdő karakterlánc kiválasztásával, az RNN állapotának inicializálásával és a létrehozandó karakterek számának beállításával kezdődik.

Megszerzi a következő karakter előrejelzési eloszlását a kezdő karakterlánc és az RNN állapot felhasználásával.

Ezután egy kategorikus eloszlás segítségével kiszámítja a megjósolt karakter indexét. Használjuk ezt a megjósolt karaktert a modell következő bemeneteként.

A modell által visszaküldött RNN-állapotot visszatápláljuk a modellbe, így az most már több kontextussal rendelkezik, nem csak egy karakterrel. A következő karakter előrejelzése után a módosított RNN-állapotokat ismét visszatápláljuk a modellbe, így az tanul, mivel több kontextust kap a korábban megjósolt karakterekből.


In [ ]:
def generate_text(model, start_string):

    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)


    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) #1-es dimenziókat eltávolítja
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"A"))

A
- Radzeer!
Holdvilág Charley valamit a Szárítónakon lengra-táljon.
- Nem is szörny... Tompaneppel vagyok? - kérdezte lehangoltan őfelsége, és megnézte magánkal. Később feleslegessé is vált a keresése, mert a herceg a telt idomú, fekete nőkre való hivatkozással.
Fernandez régens herceg körül teljesítette, és ezzel, amíg kifülé.
Mint a fenékbe mentem egy potyautast jól meg­mérgesíteni.
- Igazán?
- Igen. Volt egy utast a közelben tartózkodó hajók nevezett.
- Helyes miért ahol rántotta. Ha Menasze- Alvarez ellen hirtelen gurultot, raszít valaki olyasmesen néhány csengésű hangon a fiatalember. - Én St. Antonio uralkodó herceg vagyok. Aki madába ül!
- Az angol kilég valamit.
- Nem kínáltam!
- Öten. - Mi új pincér és Hutchins fűtő egyazon ember, akkor engem is kidobnak a csalásért.
- Ne szőr Ekmont még maratt és mosolyogva kezet fogott -, megölni, és addig szerettem, amíg elhidegült től fel karján a fiút, mintha pehely lenne.
- Ez a fiú üzenettel jött. Börtönbb doboz pincér nes volt a közel

A legegyszerűbb dolog, amit tehetsz az eredmények javítása érdekében, hogy hosszabb ideig edzed (próbáld ki az `EPOCHS = 30`).

Kísérletezhet más kezdő stringgel is, megpróbálhat egy másik RNN réteget hozzáadni a modell pontosságának javítása érdekében, vagy beállíthatja a hőmérséklet paramétert, hogy több vagy kevesebb véletlenszerű előrejelzést generáljon.

In [ ]:
text =[]
input_eval = [char2idx[s] for s in 'A']
input_eval = tf.expand_dims(input_eval, 0)
predictions = model(input_eval)

In [ ]:
predictions = tf.squeeze(predictions, 0)

In [ ]:
predictions[0,1]

<tf.Tensor: shape=(), dtype=float32, numpy=6.473366>

In [ ]:
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

In [ ]:
input_eval = tf.expand_dims([predicted_id], 0)
text.append(idx2char[predicted_id])